# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
#!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [7]:
#Replace None by your own value and justify -> 0.7 is a balanced output between 0 an 1
llm = ChatOpenAI(temperature=0.7)


/var/folders/0v/2jlpljqd1msbtz5s_xyw4yj00000gn/T/ipykernel_61930/1175458662.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.7)


In [8]:
#Write a query that would take a variable to describe any product
prompt = ChatPromptTemplate.from_template(
    "Describe the product '{product}' in detail, including its features, use cases, and target audience."
)


In [9]:

chain = LLMChain(llm=llm, prompt=prompt)

/var/folders/0v/2jlpljqd1msbtz5s_xyw4yj00000gn/T/ipykernel_61930/546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [10]:
product = "Wireless Earbuds" #Select a product type to be describe
chain.run(product)

/var/folders/0v/2jlpljqd1msbtz5s_xyw4yj00000gn/T/ipykernel_61930/2104496393.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'Wireless earbuds are a type of audio device that allows users to listen to music, podcasts, or make phone calls without the hassle of cords or wires. They are small, lightweight, and designed to fit comfortably in the ears.\n\nFeatures of wireless earbuds typically include Bluetooth connectivity, allowing them to pair with a smartphone, tablet, or other Bluetooth-enabled device. They also often include touch controls for easy operation, as well as built-in microphones for taking calls or using voice assistants.\n\nWireless earbuds are versatile and can be used in a variety of situations. They are ideal for listening to music while exercising, commuting, or working, as they provide a hands-free and tangle-free listening experience. They are also popular for making phone calls and attending virtual meetings, as they offer clear audio and noise-canceling features.\n\nThe target audience for wireless earbuds is broad, as they appeal to anyone who enjoys listening to audio on the go. Howev

## SimpleSequentialChain

In [11]:
from langchain.chains import SimpleSequentialChain

In [12]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
   "Describe the product '{product}' in detail, including its features, use cases, and target audience."
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [13]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Give multiple examples of  '{product}'"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [14]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [15]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Wireless earbuds are small, lightweight electronic devices that are designed to be worn in the ears without the need for any cords or wires connecting them to a device. They use Bluetooth technology to wirelessly connect to devices such as smartphones, tablets, or laptops, allowing users to listen to music, take phone calls, or even interact with voice assistants without the restrictions of traditional wired headphones.

Some key features of wireless earbuds include:

1. Portability: Wireless earbuds are compact and easy to carry around, making them ideal for on-the-go use.
2. Convenience: Users do not have to worry about tangled cords or getting caught on objects while using wireless earbuds.
3. Battery life: Most wireless earbuds come with a charging case that provides additional battery life on the go.
4. Noise cancellation: Many models offer noise-canceling technology to block out ambient noise and provide a more immersive listening e

'Examples of wireless earbuds on the market include:\n\n1. Apple AirPods: One of the most popular wireless earbuds on the market, known for their seamless integration with Apple devices and long battery life.\n2. Samsung Galaxy Buds: Wireless earbuds designed to work with Samsung smartphones and tablets, with features like ambient sound mode and touch controls.\n3. Jabra Elite 75t: A pair of wireless earbuds known for their comfortable fit, long battery life, and customizable sound settings.\n4. Bose QuietComfort Earbuds: Premium wireless earbuds with noise-canceling technology for a more immersive audio experience.\n5. Sony WF-1000XM4: High-end wireless earbuds with industry-leading noise-canceling technology and excellent sound quality.'

**Repeat the above twice for different products**

## SequentialChain

In [30]:
from langchain.chains import SequentialChain

In [31]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review into German:\n{review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="translate_review" #Give a name to your output
                    )


In [32]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review:\n{translate_review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summarize_review" #give a name to this output
                    )


In [34]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate the following review into: Russian\n{summarize_review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="translated_text"
                      )


In [40]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
    "Based on the following summary:\n"
    "{summarize_review}\n"
    "And the translation:\n"
    "{translated_text}\n"
    "Write a follow-up message that is engaging and relevant."
)

chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="follow_up_message"
                     )


In [42]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],  # Add your chains
    input_variables=["review"],  # Inputs for the first chain
    output_variables=["follow_up_message","translated_text","summarize_review"], # Final output
    verbose=True
)

In [43]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'follow_up_message': "Thank you for your feedback on the product. We apologize for the poor taste and quality you experienced. We assure you that our products are not old or fake, but we will definitely look into this issue further to ensure it doesn't happen again. We value your input and hope to have the opportunity to provide you with a better experience in the future. Thank you for bringing this to our attention.",
 'translated_text': 'Рецензент нашел вкус продукта плохим и отметил, что пена плохо держится. Они упомянули, что обычно покупают тот же продукт в магазинах и находят вкус намного лучше, задавая вопрос, не старый ли это товар или фальшивка.',
 'summarize_review': 'The reviewer found the taste of the product to be poor and noted that the foam did not hold well. They mentioned that they usually buy

**Repeat the above twice for different products or reviews**

## Router Chain

In [44]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [45]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [46]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [47]:
llm = ChatOpenAI(temperature=0)

In [48]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [49]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [50]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [51]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [52]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

/var/folders/0v/2jlpljqd1msbtz5s_xyw4yj00000gn/T/ipykernel_61930/3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [53]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases. Black body radiation plays a key role in understanding concepts such as thermal radiation and the quantization of energy in quantum mechanics."

In [54]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [55]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'

**Repeat the above at least once for different inputs and chains executions - Be creative!**